In [25]:
# This is a cell to hide code snippets from displaying
# This must be at first cell!

from IPython.display import HTML

hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).hide();
      }
    });
    $('div.output_prompt').css('opacity', 0);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input style="opacity:0" type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [26]:
hide_me

from metakernel import register_ipython_magics
register_ipython_magics()

import ipywidgets as widgets
import sys
from IPython.display import display
from IPython.display import clear_output

from pythonds.basic import Stack


def create_multipleChoice_widget(description, options, correct_answer):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    radio_options = [(words, i) for i, words in enumerate(options)]
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False
    )
    
    description_out = widgets.Output()
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "Richtig." + '\x1b[0m' +"\n" #green color
        else:
            s = '\x1b[5;30;41m' + "Fail. " + '\x1b[0m' +"\n" #red color
        with feedback_out:
            clear_output()
            print(s)
        return
    
    check = widgets.Button(description="submit")
    check.on_click(check_selection)
    
    
    return widgets.VBox([description_out, alternativ, check, feedback_out])


frage1 = 'Welches I/O-Prinzip liegt der Datenstruktur Stack zugrunde'
frage2 ='Was ist das Ergebnis bei folgenden Operationen? m = Stack(), m.push(x), m.push(y), m.pop(), m.push(z), m.peek()'
frage3 ="""Was ist das Ergebnis bei folgenden Operationen?

m = Stack()
m.push(x)
m.push(y)
m.push(z)
while not m.isEmpty():
   m.pop()
   m.pop()
m.peek()
"""

Q1 = create_multipleChoice_widget(frage1,['First-In, First-Out (FIFO)','Last-In, First-Out (LIFO)','Last-Out, First-In (LOFI)'],'Last-In, First-Out (LIFO)')
Q2 = create_multipleChoice_widget(frage2,['x','y','z','Der Stapel ist leer'],'z')
Q3 = create_multipleChoice_widget(frage3, ['x','der Stapel ist leer','ein Fehler erscheint','z'],'ein Fehler erscheint')


<img src="Bilder/MIREVIBanner.jpg">

# Abstrakte Datenstrukturen, Teil 1

---
## Ziele 
- die abstrakten Datentypen Stack, Queue, Deque und List verstehen.
- die ADTs Stack, Queue und Deque unter Verwendung von Python-Listen implementieren können.
- die Performanz der Implementierungen von grundlegenden linearen Datenstrukturen zu verstehen.
- Verstehen der Präfix-, Infix- und Postfix-Ausdrucksformate.
- Verwendung von Stacks zur Auswertung von Postfix-Ausdrücken.
- Verwendung von Stacks zur Konvertierung von Ausdrücken von Infix- in Postfix-Ausdrücke.
- Verwendung von Warteschlangen für grundlegende Timing-Simulationen.
- Probleme erkennen, bei denen Stacks, Warteschlangen und Deques geeignete Datenstrukturen sind.
- den ADT Liste als verlinkte Liste unter Verwendung des Knoten- und Zeigerkonzepts implementieren zu können.
- die Leistung der Implementierung der verknüpften Liste mit der Listenimplementierung von Python vergleichen.
---
### Motivation
Wir beginnen unser Studium der Datenstrukturen mit der Betrachtung von vier einfachen, aber sehr mächtigen Konzepten: *Stapel, Warteschlangen, Deques* und *Listen* sind Beispiele für Datensammlungen, deren Elemente je nachdem, wie sie hinzugefügt oder entfernt werden, geordnet sind. Sobald ein Element hinzugefügt wird, bleibt es in dieser Position relativ zu den anderen Elementen, die davor und danach kamen. Sammlungen wie diese werden oft als lineare Datenstrukturen bezeichnet.

Lineare Strukturen kann man sich vorstellen, dass sie zwei Endpunkte haben. Manchmal werden diese Enden als das "linke" und das "rechte" oder in einigen Fällen als das "vordere" und das "hintere" Ende bezeichnet. Man könnte sie auch als "oben" und "unten" bezeichnen. Die verschiedenen Namen, die den Enden gegeben werden, sind nicht entscheidend. Was eine lineare Struktur von einer anderen unterscheidet, ist die Art und Weise, in der Elemente hinzugefügt und entfernt werden, insbesondere die Stelle, an der diese Hinzufügungen und Entfernungen stattfinden. Zum Beispiel könnte eine Struktur es erlauben, dass neue Elemente nur an einem Ende hinzugefügt werden. Einige Strukturen könnten es erlauben, Elemente an beiden Enden zu entfernen.

Aus diesen Variationen ergeben sich einige der nützlichsten Datenstrukturen in der Informatik. Sie kommen in vielen Algorithmen vor und können zur Lösung einer Vielzahl von wichtigen Problemen verwendet werden.

---
## Stack, Keller und Stapel
Ein Stapel (manchmal auch als "Push-Down-Stapel", Stack oder Keller bezeichnet) ist eine geordnete Sammlung von Elementen, bei der das Hinzufügen neuer Elemente und das Entfernen bestehender Elemente immer am gleichen Ende stattfindet. Dieses Ende wird allgemein als *oben* bezeichnet. Das dem oberen Ende gegenüberliegende *untere* Ende wird als "Basis" bezeichnet.

Das untere Ende des Stapels ist von Bedeutung, da die näher am unteren Ende des Stapels gelagerten Artikel diejenigen repräsentieren, die am längsten auf dem Stapel liegen. Der zuletzt hinzugefügte Gegenstand ist derjenige, der sich in der Position befindet, zuerst entfernt zu werden. Dieses Ordnungsprinzip wird manchmal als **LIFO, last-in-first-out**, bezeichnet. Es bietet eine Ordnung, die auf der Länge der Zeit in der Sammlung basiert. Neuere Artikel befinden sich in der Nähe der Spitze, während ältere Artikel sich in der Nähe der Basis befinden.

Viele Beispiele von Stapeln kommen in alltäglichen Situationen vor. Fast jede Cafeteria hat einen Stapel von Tabletts oder Tellern, bei dem man das oberste Tablett oder den Teller nimmt und so ein neues Tablett oder einen neuen Teller für den nächsten Kunden in der Reihe aufdeckt. Stellen Sie sich einen Stapel Bücher auf einem Schreibtisch vor (Abbildung 1). Das einzige Buch, dessen Einband sichtbar ist, ist das oberste. Um auf die anderen Bücher des Stapels zugreifen zu können, müssen wir die Bücher, die darauf liegen, entfernen. Abbildung 2 zeigt einen weiteren Stapel. Dieser enthält eine Reihe primitiver Python-Datenobjekte.

**Abb. 1 Stack als abstrakte Datenstruktur (ADT)**
<div>
<img src="Bilder/ADT/bookstack2.png" width=300px style="margin:5px" align="left">
<img src="Bilder/ADT/primitive.png"  width=300px align="left"> 
</div>
<br>
</br>

Eine der nützlichsten Ideen im Zusammenhang mit Stapeln ergibt sich aus der einfachen Beobachtung der Gegenstände, während sie hinzugefügt und dann wieder entfernt werden. Angenommen, Sie beginnen mit einem leeren Schreibtisch. Nun legen Sie die Bücher eins nach dem anderen übereinander. Sie konstruieren einen *Stapel*. Überlegen Sie, was passiert, wenn Sie beginnen, Bücher zu entfernen. Die Reihenfolge, in der sie entfernt werden, ist genau umgekehrt zu der Reihenfolge, in der sie platziert wurden. Stapel sind von grundlegender Bedeutung, da sie verwendet werden können, um die Reihenfolge der Elemente umzukehren. Die Reihenfolge des Einfügens ist die umgekehrte der Reihenfolge des Herausnehmens. *Abbildung 3* zeigt den Python-Datenobjekt-Stapel, wie er erstellt wurde und dann wieder wie die Elemente entfernt werden. Beachten Sie die Reihenfolge der Objekte.

In Anbetracht dieser Umkehreigenschaft fallen Ihnen vielleicht Beispiele von Stapeln ein, die bei der Benutzung Ihres Computers auftreten. Zum Beispiel hat jeder Webbrowser eine Zurück-Schaltfläche. Wenn Sie von Web-Seite zu Web-Seite navigieren, werden diese Seiten auf einem Stapel abgelegt (eigentlich sind es die URLs, die auf den Stapel gelegt werden). Die aktuelle Seite, die Sie betrachten, befindet sich ganz oben und die erste Seite, die Sie sich angesehen haben, ganz unten. Wenn Sie auf die Schaltfläche Zurück klicken, beginnen Sie, sich in umgekehrter Reihenfolge durch die Seiten zu bewegen.

Der abstrakte Stack-Datentyp wird durch die folgende Struktur und die folgenden Operationen definiert. Ein Stapel ist, wie oben beschrieben, als eine geordnete Sammlung von Elementen strukturiert, wobei Elemente am Ende, dem so genannten "Anfang", hinzugefügt und entfernt werden. Stapel sind LIFO-geordnet. Die Stapel-Operationen sind unten angegeben.

- <code>stack()</code> erzeugt einen neuen Stapel, der leer ist. Es benötigt keine Parameter und gibt einen leeren Stapel zurück.
- <code>push(item)</code> fügt einen neuen Artikel an die Spitze des Stapels hinzu. Es benötigt den Artikel und gibt nichts zurück.
- <code>pop()</code> entfernt das oberste Element vom Stapel. Es benötigt keine Parameter und gibt das Element zurück. Der Stapel wird geändert.
- <code>peek()</code> gibt das oberste Element vom Stapel zurück, entfernt es aber nicht. Es benötigt keine Parameter. Der Stapel wird nicht verändert.
- <code>isEmpty()</code> prüft, ob der Stapel leer ist. Sie benötigt keine Parameter und gibt einen booleschen Wert zurück.
- <code>size()</code> gibt die Anzahl der Elemente auf dem Stapel zurück. Sie benötigt keine Parameter und gibt eine ganze Zahl zurück.

Wenn $s$ beispielsweise ein Stapel ist, der erstellt wurde und leer beginnt, dann zeigt Tabelle 1 die Ergebnisse einer Folge von Stapeloperationen. Unter Stapelinhalt wird das oberste Element ganz rechts aufgelistet.

<img src="Bilder/ADT/StackBefehle.jpg" width=800px>

---
### Aufgabe ADT-01: Kellerinhalt erzeugen
Beschreiben Sie die minimale Anzahl an Operationen mit denen man von einem Keller A zum Keller B kommt. Der Anfang des Kellers (das oberste Element) ist *rechts*.

<table width=250px style="margin:5px" align="center">
  <thead>
    <tr>
      <th>A</th>
      <th>B</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>[]</td>
      <td>[a,b,c,d,e]</td>
    </tr>
    <tr>
      <td>[1,2,3,4,5]</td>
      <td>[2,4,6]</td>
    </tr>
      <tr>
      <td>[1,2,3,4,5,6]</td>
      <td>[6,5,4,3,2,1]</td>
    </tr>
  </tbody>
</table>

---

## Stack (Keller) in Python implementieren

Da wir den Stack als abstrakten Datentyp definiert haben, werden wir unsere Aufmerksamkeit auf die Verwendung von Python zur Implementierung des Stacks richten. Erinnern Sie sich, dass wir, wenn wir einem abstrakten Datentyp eine physische Implementierung geben, die Implementierung als *Datenstruktur* bezeichnen.

Wie wir in Kapitel 1 beschrieben haben, ist in Python, wie in jeder objektorientierten Programmiersprache, die beste Implementierung für einen abstrakten Datentyp wie einen Stack die Erstellung einer neuen *Klasse*. Die Stack-Operationen werden als *Methoden* implementiert. Um einen Stack zu implementieren, bei dem es sich um eine Sammlung von Elementen handelt, ist es außerdem sinnvoll, die Leistungsfähigkeit und Einfachheit der einfachen  "Collections" von Python zu nutzen. Wir werden das Modul *Liste* verwenden.

Erinnern wir uns, dass die Klasse <code>list</code> in Python eine Reihe geeigneter Methoden bereitstellt. Wenn wir zum Beispiel die Liste haben, brauchen wir nur zu entscheiden, welches Ende der Liste als oberstes und welches als unterstes Ende des Stapels betrachtet wird. Sobald diese Entscheidung getroffen ist, können die Operationen mit den Listenmethoden wie ``append`` und ``pop`` implementiert werden.

Die folgende Stack-Implementierung geht davon aus, dass das Ende der Liste das oberste Element des Stacks enthält. Wenn der Stapel wächst (bei `push`-Operationen), werden neue Elemente am Ende der Liste hinzugefügt. `pop`-Operationen manipulieren dasselbe Ende.

**Listing 1**

In [32]:
class Stack:
     def __init__(self):
         self.items = []

     def isEmpty(self):
         return self.items == []

     def push(self, item):
         self.items.append(item)

     def pop(self):
         return self.items.pop()

     def peek(self):
         return self.items[len(self.items)-1]

     def size(self):
         return len(self.items)

s=Stack()
print(s.isEmpty())

True


*Listing 1* zeigt die Klasse Stack in Aktion, während wir die Reihenfolge der Operationen aus Tabelle 1 ausführen. 

---
### Aufgabe ADT-02: Kellerinhalt aus ADT-01 generieren
Erzeugen Sie in **Listing 1** die in der Aufgabe **ADT-01** von Ihnen entwickelten Lösungen und überprüfen Sie ob die korrekten Inhalte des Stack generiert werden


**Listing 2** Aufgabe ADT-02

In [33]:
# Stackobjekt s1 erzeugen

s1=Stack()

#Operationen Ihrer Lösung 1 anwenden

#Ausgabe
print(s1)

# Stackobjekt s1 erzeugen

s2=Stack()

#Operationen Ihrer Lösung 2 anwenden

#Ausgabe
print(s2)


# Stackobjekt s3 erzeugen

s3=Stack()

#Operationen Ihrer Lösung 3 anwenden

#Ausgabe
print(s3)


### Ergänzende Bemerkung###
An dieser Stelle sei zusätzlich bemerkt, dass wir den Stack mit einer Liste hätten implementieren können, bei der der "Anfang" am Anfang und nicht am Ende steht. In diesem Fall würden die bisherigen Pop- und Append-Methoden nicht mehr funktionieren und wir müssten die Position 0 (das erste Element in der Liste) explizit mit Pop und Insert indizieren. Die Implementierung ist in Listing dargestellt.

**Listing 3.** *Alternative Implementation of the Stack class (stack_cl_1)*

In [35]:
class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
         return self.items == []

    def push(self, item):
         self.items.insert(0,item)

    def pop(self):
         return self.items.pop(0)

    def peek(self):
        return self.items[0]

    def size(self):
        return len(self.items)

s = Stack()
s.push('hello')
s.push('true')
print(s.pop())


true


Man erkennt hier die Stärke des Konzepts eines abstrakten Datentyps und die Implementierung mit einer objektorientierte Programmiersprache. Wir können die Implementierung ändern, die Schnittstelle zur Nutzung ändert sich jedoch nicht wesentlich.

---
### ADT-03 Selbstest
Bearbeiten Sie die drei Fragen und die Programmieraufgabe.

In [37]:
hide_me


display(Q1)
display(Q2)
display(Q3)


4. Programmieraufgabe: Schreiben Sie eine Funktion ``revstring(mystr)``, die einen Stack verwendet, um die Zeichen in einer Zeichenfolge umzukehren.

```
def revstr(mystr):
    # Erzeuge Stack
    # Speichere jeden Buchstaben auf Stack
    # Erzeuge leeren String revstr
    # Hänge jeden Eintrag des Stack an revstr 
    # gib revstr zurück
#print(revstr("Hallo Hallo"))
```